In [1]:
"""
Feature: heart_rate
Extract max heart rate from chartevents (ICU) and vitalsign (ED)
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load ICU chartevents in chunks
heart_itemids = [220045, 220046, 220047]
heart_max_list = []

try:
    chunk_iter = pd.read_csv(
        os.path.join(icu_path, "chartevents.csv"),
        usecols=["subject_id", "hadm_id", "itemid", "valuenum"],
        chunksize=1000000,
        low_memory=False
    )
    
    for chunk in chunk_iter:
        hr = chunk[chunk["itemid"].isin(heart_itemids)].copy()
        hr = hr.dropna(subset=["valuenum"])
        hr = hr[(hr["valuenum"] >= 20) & (hr["valuenum"] <= 300)]
        
        if not hr.empty:
            hr_max_chunk = hr.groupby("hadm_id")["valuenum"].max()
            heart_max_list.append(hr_max_chunk)
    
    if heart_max_list:
        heart_max_icu = (
            pd.concat(heart_max_list)
            .groupby("hadm_id")
            .max()
            .reset_index()
            .rename(columns={"valuenum": "heart_rate_icu"})
        )
    else:
        heart_max_icu = pd.DataFrame(columns=["hadm_id", "heart_rate_icu"])
except:
    heart_max_icu = pd.DataFrame(columns=["hadm_id", "heart_rate_icu"])

# Load ED vitalsign
try:
    vitalsign = pd.read_csv(os.path.join(ed_path, "vitalsign.csv"))
    ed_stays = pd.read_csv(os.path.join(ed_path, "edstays.csv"))
    
    ed_vitals_hr = ed_stays.merge(
        vitalsign[["subject_id", "stay_id", "heartrate"]],
        on=["subject_id", "stay_id"],
        how="left"
    )
    
    heart_max_ed = (
        ed_vitals_hr.groupby(["subject_id", "hadm_id"])["heartrate"]
        .max()
        .reset_index()
        .rename(columns={"heartrate": "heart_rate_ed"})
    )
except:
    heart_max_ed = pd.DataFrame(columns=["subject_id", "hadm_id", "heart_rate_ed"])

# Merge
df = df.merge(heart_max_icu, on="hadm_id", how="left")
if not heart_max_ed.empty:
    df = df.merge(heart_max_ed, on=["subject_id", "hadm_id"], how="left")

# Combine: prefer ICU, then ED
df["heart_rate"] = df["heart_rate_icu"].fillna(df.get("heart_rate_ed", pd.Series()))

# Drop intermediate columns
df.drop(columns=["heart_rate_icu", "heart_rate_ed"], errors="ignore", inplace=True)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'heart_rate' added. Non-null count: {df['heart_rate'].notna().sum()}")
print(f"Dataset shape: {df.shape}")



Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 94)
Feature 'heart_rate' added. Non-null count: 111000
Dataset shape: (158020, 94)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [4]:
# cols_to_drop = [
#  "heart_rate"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)